In [1]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA

In [2]:
from langchain_community.document_loaders import DirectoryLoader

In [3]:
# Load environment variables
from dotenv import load_dotenv
import os

load_dotenv()

nvidia_api_key = os.getenv('NVIDIA_API_KEY')

if nvidia_api_key is None:
    raise ValueError("NVIDIA_API_KEY not found in .env file")

os.environ['NVIDIA_API_KEY'] = nvidia_api_key

print("NVIDIA_API_KEY loaded successfully.")

NVIDIA_API_KEY loaded successfully.


In [4]:
import os

os.environ["GROQ_API_KEY"] = "gsk_..."
os.environ["TOKENIZERS_PARALLELISM"] = "false" # To suppress huggingface warnings

In [5]:
## Necessary for Colab, not necessary for course environment
# %pip install -qq langchain langchain-nvidia-ai-endpoints gradio


# import os
# os.environ["NVIDIA_API_KEY"] = "nvapi-..."

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [6]:
from langchain_core.runnables import RunnableLambda
from functools import partial

def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

In [7]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# Load Markdown documents
loader = DirectoryLoader(
    './functions', 
    glob="**/*.md", 
    loader_cls=TextLoader,
    show_progress=True,
    use_multithreading=True,
    loader_kwargs={'autodetect_encoding': True}
)

# Load documents
docs = loader.load()

100%|██████████| 329/329 [00:00<00:00, 545.05it/s]


In [8]:
# Print sample documents after loading
print("Sample loaded documents:")
for doc in docs[:3]:
    print(f"Content: {doc.page_content[:500]}")  # Print the first 500 characters
    print(f"Metadata: {doc.metadata}\n")


Sample loaded documents:
Content: ---
id: '1DIVVSjNfv48-LekRsKDHFuHEm1gBYphsCC18iR2WikU'
title: 'E-Chart'
date: '2020-02-27T19:44:20.933Z'
version: 24
mimeType: 'text/x-markdown'
links: []
source: 'https://drive.google.com/open?id=1DIVVSjNfv48-LekRsKDHFuHEm1gBYphsCC18iR2WikU'
wikigdrive: 'dd69069d725fca5f553df7ded62e130a49d49ca6'
---
The electronic chart, or E-Chart, is an essential component of the {{% system-name %}} solution. It allows for a single, integrated view of all scanned and electronic documents, as well as all recor
Metadata: {'source': 'functions\\e-chart.md'}

Content: ---
id: '1WybRBSEox5cSECcPwj3PJOArnmlFKwVMkNlT6gq1qSQ'
title: 'Document Management'
date: '2020-02-27T21:09:07.746Z'
version: 42
lastAuthor: 'aquandt'
mimeType: 'text/x-markdown'
links:
  - 'document-management/documents-and-forms.md'
  - 'document-management/scanning-and-indexing.md'
  - 'document-management/printing-and-print-definitions.md'
  - 'document-management/imaging.md'
source: 'https://drive.goo

In [9]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter


# Define headers to split on
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

# Initialize the MarkdownHeaderTextSplitter
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Apply the markdown splitter to the text content of each document
md_header_splits = []
for doc in docs:
    splits = markdown_splitter.split_text(doc.page_content)
    for split in splits:
        md_header_splits.append(Document(page_content=split.page_content, metadata=doc.metadata))

# Print sample documents after splitting by markdown headers
print("\nSample documents after splitting by markdown headers:")
for doc in md_header_splits[:3]:
    print(f"Content: {doc.page_content[:500]}")  # Print the first 500 characters
    print(f"Metadata: {doc.metadata}\n")

# Initialize the RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

# Chunk the split documents
chunked_docs = text_splitter.split_documents(md_header_splits)




Sample documents after splitting by markdown headers:
Content: ---
id: '1DIVVSjNfv48-LekRsKDHFuHEm1gBYphsCC18iR2WikU'
title: 'E-Chart'
date: '2020-02-27T19:44:20.933Z'
version: 24
mimeType: 'text/x-markdown'
links: []
source: 'https://drive.google.com/open?id=1DIVVSjNfv48-LekRsKDHFuHEm1gBYphsCC18iR2WikU'
wikigdrive: 'dd69069d725fca5f553df7ded62e130a49d49ca6'
---
The electronic chart, or E-Chart, is an essential component of the {{% system-name %}} solution. It allows for a single, integrated view of all scanned and electronic documents, as well as all recor
Metadata: {'source': 'functions\\e-chart.md'}

Content: ---
id: '1WybRBSEox5cSECcPwj3PJOArnmlFKwVMkNlT6gq1qSQ'
title: 'Document Management'
date: '2020-02-27T21:09:07.746Z'
version: 42
lastAuthor: 'aquandt'
mimeType: 'text/x-markdown'
links:
- 'document-management/documents-and-forms.md'
- 'document-management/scanning-and-indexing.md'
- 'document-management/printing-and-print-definitions.md'
- 'document-management/imaging.md'
sour

In [6]:
embeddings = NVIDIAEmbeddings()

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.vectorstores import FAISS


vector = FAISS.from_documents(chunked_docs, embeddings)
retriever = vector.as_retriever


NameError: name 'chunked_docs' is not defined

In [8]:
query = "What functionalities does the document management system provide"
# docs = db.similarity_search(query)
retriever = vector.as_retriever()
docs = retriever.invoke(query)
print (docs)

NameError: name 'vector' is not defined

In [16]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS



model = ChatNVIDIA(model="meta/llama3-70b-instruct")

In [17]:
model

ChatNVIDIA(model='meta/llama3-70b-instruct')

In [18]:
hyde_template = """Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:

{question}"""
hyde_prompt = ChatPromptTemplate.from_template(hyde_template)
hyde_query_transformer = hyde_prompt | model | StrOutputParser()

In [19]:
from langchain_core.runnables import chain

@chain
def hyde_retriever(question):
    hypothetical_document = hyde_query_transformer.invoke({"question": question})
    return retriever.invoke(hypothetical_document)

In [20]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
answer_chain = prompt | model | StrOutputParser()

In [21]:
@chain
def final_chain(question):
    documents = hyde_retriever.invoke(question)
    for s in answer_chain.stream({"question": question, "context": documents}):
        yield s

In [22]:

for s in final_chain.stream("What functionalities does the system provide for managing patient flow"):
    print(s, end="")

Based on the provided context, the system provides the following functionalities for managing patient flow:

1. Initiation of patient encounters
2. Notifications for clinical staff of patients' locations
3. Tracking of patient wait times
4. Moving patients between varying stations or locations through to checkout using the Station Manager feature
5. Color-coded station alerts to notify users when a patient has remained at a station for an extended period of time
6. Fast and easily-navigated functionality for searching current charts and documents
7. Functionality for registering new patients
8. Automated checkout of patients not checked out by staff at the end of the day (configurable)

In [23]:
for s in final_chain.stream("How does the document management system improve client experience?"):
    print(s, end="")

According to the provided context, the document management system improves client experience by providing "easy chart access, anywhere, anytime" and "hassle-free interoperability and integrated workflows, intended to streamline operations and communication."

In [24]:
for s in final_chain.stream("What functionalities does the E-Sign module provide?"):
    print(s, end="")

According to the provided context, the E-Sign module allows for:

1. Adding documents
2. Adding tasks
3. Adding dictations
4. Requesting co-signers or multiple signatures as needed.

It is also mentioned that the E-Sign module has a "Pending Reviews" tab, but the exact functionality of this tab is not specified.

In [25]:
for s in final_chain.stream("How does the system support telehealth?"):
    print(s, end="")

According to the provided context, the system supports telehealth in the following ways:

1. Rapid addition of telehealth capability to the system, including audio and video capabilities.
2. Patients can join a video visit from a portal branded to the clinic or practice, using a computer equipped with a camera or a smartphone or mobile device.
3. The system's telehealth functionality adheres to guidance provided by the U.S. Department of Health and Human Services.

Overall, the system enables clinics and practices to manage telehealth visits while using a fully-certified electronic health record.

In [26]:
# Enhanced HYDE Template
hyde_template = """Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question. Make sure to stay relevant to the topic and include plausible details that align with common functionalities in document management systems:

{question}"""
hyde_prompt = ChatPromptTemplate.from_template(hyde_template)
hyde_query_transformer = hyde_prompt | model | StrOutputParser()


In [27]:
# Define the chain
from langchain_core.runnables import chain

@chain
def hyde_retriever(question):
    hypothetical_document = hyde_query_transformer.invoke({"question": question})
    return retriever.invoke(hypothetical_document)

In [28]:
# Enhanced Main Template
template = """Answer the question based only on the following context. Ensure your answer is precise, accurate, and directly addresses the question:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
answer_chain = prompt | model | StrOutputParser()

In [29]:
@chain
def final_chain(question):
    documents = hyde_retriever.invoke(question)
    for s in answer_chain.stream({"question": question, "context": documents}):
        yield s

In [30]:
# Test the chains with different questions
questions = [
    "What functionalities does the system provide for managing patient flow?",
    "How does the document management system improve client experience?",
    "What functionalities does the E-Sign module provide?",
    "How does the system support telehealth?"
]

In [31]:
for question in questions:
    print(f"\nQuestion: {question}")
    for s in final_chain.stream(question):
        print(s, end="")
    print("\n")


Question: What functionalities does the system provide for managing patient flow?
According to the provided context, the system provides the following functionalities for managing patient flow:

1. Initiation of patient encounters
2. Notifications for clinical staff of patients' locations
3. Tracking of patient wait times
4. Moving patients between varying stations or locations through to checkout using the Station Manager
5. Color-coded station alerts for extended patient wait times
6. Fast and easily-navigated functionality for searching current charts and documents
7. Registering new patients
8. One-Click Checkin to quickly check in patients from the Scheduler
9. Evaluating patient flow through check-in and appointment statistics and reports with filtering options (by resource, station, location, patient, or date range) and printing or tasking reports to administrators for review.


Question: How does the document management system improve client experience?
According to the provid

In [33]:
import faiss
import pickle

# Save the FAISS index
faiss.write_index(vector.index, "faiss_index.bin")

# Save the metadata
with open("faiss_metadata.pkl", "wb") as f:
    pickle.dump(vector.docstore, f)

In [39]:
## Save and compress your index
vector.save_local("FAISS_index")
!tar czvf vector.tgz vectorstore_index

!rm -rf vectorstore_index

tar: : Couldn't visit directory: No such file or directory
tar: Error exit delayed from previous errors.
'rm' is not recognized as an internal or external command,
operable program or batch file.


In [40]:
import os
import tarfile
from langchain_community.vectorstores import FAISS

# Save the FAISS vector store
vector.save_local("FAISS_index")

# Compress the saved vector store directory into a tar.gz file
with tarfile.open("vectorstore_index.tgz", "w:gz") as tar:
    tar.add("FAISS_index", arcname=os.path.basename("FAISS_index"))

# Remove the original directory after compressing it
import shutil
shutil.rmtree("FAISS_index")
print("FAISS index saved and compressed successfully.")

FAISS index saved and compressed successfully.


In [43]:
import os
import tarfile
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# Decompress the tar.gz file
with tarfile.open("vectorstore_index.tgz", "r:gz") as tar:
    tar.extractall()

# Initialize the embedding model
# embeddings = NVIDIAEmbeddings(model="ai-embed-qa-4")

# Load the FAISS vector store
new_db = FAISS.load_local("FAISS_index", embeddings, allow_dangerous_deserialization=True)

# Test the loaded vector store
docs = new_db.similarity_search("Testing the index")
print(docs[0].page_content[:1000])

. For more information on how to index, please see other help documentation named *Indexing.pdf*


In [46]:
import gradio as gr

def answer_question(question):
    responses = []
    for s in final_chain.stream(question):
        responses.append(s)
    return "".join(responses)

# Set up the Gradio interface
iface = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs=gr.Textbox(),
    title="Document Management System Q&A",
    description="Ask questions about the functionalities of the Document Management System."
)

# Launch the interface
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://702eb60f5d9bcb5ae5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
